#Student Name: Chelliah Kanthanathan
#Student ID: 18263003

In [1]:
import random
import unittest


# Class TensorNotSuitableError
# Derived from base class Exception.
# init method receive error string and print it
class TensorNotSuitableError(Exception):
    def __init__(self,error):
        self.errorstring = error

# Class Tensor
# This class includes relevant methods for tensor operations
# The main class field my_tensor that points to tensor
class Tensor:
    # initialize tensor, invoke create_tensor to create and fill the tensor
    def __init__(self,tensor_size=0):
        if (tensor_size != 0):
            self.my_tensor = self.create_tensor(tensor_size)
        else:
            self.my_tensor = []
    
    # return tensor
    def return_my_tensor(self):
        return self.my_tensor
    
    # set tensor
    def set_my_tensor(self,tensor):
        self.my_tensor = tensor
    
    # create tensor of given size. Input is tuple with two elements first 
    # element represent row , second element represent column. This method 
    # creates tensor and fill it with zeros. This method is mainly used to create 
    # empty result_tensor
    def create_all_zero_tensor(self,size):
        row = size[0]
        column = size[1]
        tensor = [0] * row
        for index in range (row):
            tensor[index] = [0] * column
        return tensor
        
    # create tensor of given size. Input is tuple with two elements first 
    # element represent row , second element represent column. This method 
    # creates tensor and fill it with random numbers
    def create_tensor(self, size):
        row = size[0]
        column = size[1]
        tensor = [0] * row
        for index in range (row):
            tensor[index] = [0] * column
            
        # populate tensor with random values
        for row_index in range(row):
            for column_index in range(column):
                tensor[row_index][column_index] = random.randint(1,8)                    
        return tensor

    # This method calculates tensor size and return it as tuple
    def calculate_tensor_size(self):
        rows = len(self.my_tensor)
        columns = len(self.my_tensor[0])
        return (rows , columns)
    
    # This method is used to print tensor in pretty way
    def format_tensor(self,tensor):
        tensor_in_string = str(tensor)
        tensor_in_string = tensor_in_string.replace('],',',\n')
        tensor_in_string = tensor_in_string.replace('\n ','\n')
        tensor_in_string = tensor_in_string.replace(',','')
        tensor_in_string = tensor_in_string.replace('[','')
        tensor_in_string = tensor_in_string.replace(']','')
        tensor_in_string = tensor_in_string.replace(' ','  ')
        return tensor_in_string
    
    # check if the provided tensor is of same size, Incase of addition or 
    # subtraction the rows and columns of two tensores should match.
    # if incompatible return true, otherwise return false
    def check_if_incompatible_tensors_for_addition_or_subtraction(self,first_tensor,second_tensor):         
        if (len(first_tensor) == len(second_tensor)):
            if(len(first_tensor[0]) == len(second_tensor[0])):
                return False
            else:
                return True
        else:
            return True
    
    # check if the provided tensor is of same size, Incase of multiplication 
    # the column of first tensor should match with row of second tensor.
    # if incompatible return true, otherwise return false
    def check_if_incompatible_tensors_for_multiplication(self,first_tensor,second_tensor):         
        if (len(first_tensor[0]) == len(second_tensor)):
            return False
        else:
            return True

    # This method accepts two tensors and perform addition or subtraction. The size 
    # of Input tensors are checked, Incase of different size then an exception is raised
    def tensor_addition_or_subtraction(self, other_tensor, operation="addition"):
        # check if the input tensors are compatible, if not raise an exception
        try:
            if (self.check_if_incompatible_tensors_for_addition_or_subtraction(self.my_tensor,other_tensor)):
                raise TensorNotSuitableError("Tensors not compatible for addition or subtraction!!!")
        except TensorNotSuitableError:
            raise
            
        # create result tensor 
        result_tensor = self.create_all_zero_tensor((len(self.my_tensor),len(other_tensor[0])))
        # get the rows of first tensor 
        for row in range (len(self.my_tensor)):
            column = 0
            # get the column of second tensor 
            for i, j in zip(self.my_tensor[row],other_tensor[row]):
                if (operation == "subtraction"):
                    result_tensor[row][column] = i - j
                else:
                    result_tensor[row][column] = i + j
                column += 1
        return result_tensor

    # This method accepts two tensors and perform multiplication. The size of 
    # Input tensors are checked, Incase of different size then an exception is raised
    def tensor_multiplication(self,other_tensor):
        # check if the input tensors are compatible, if not raise an exception
        try:
            if (self.check_if_incompatible_tensors_for_multiplication(self.my_tensor,other_tensor)):
                raise TensorNotSuitableError("Tensors not compatible for multiplication!!!")
        except TensorNotSuitableError:
            raise
            
        # create result tensor 
        result_tensor = self.create_all_zero_tensor((len(self.my_tensor),len(other_tensor[0])))
        # get the rows of first tensor
        for row in range(len(self.my_tensor)):
            # get the columns of second tensor
            for column in range(len(other_tensor[0])):
                # get the rows of second tensor
                for row2 in range(len(other_tensor)):
                    result_tensor[row][column] += self.my_tensor[row][row2] * other_tensor[row2][column]
        return result_tensor

    # This method calculate determinant of 2x2 tensor
    def tensor_determinant(self):
        # check if the tensor is compatible for determinant calcualtion, if not raise an exception
        # tensor as a square matrix with size 2x2 is compatible for determinanat calculation
        # note as of now only 2x2 tensor is supported
        try:
            if(len(self.my_tensor) != 2 or len(self.my_tensor[0]) != 2):
                raise TensorNotSuitableError("Tensor not compatible for determinant!!!")
        except TensorNotSuitableError:
            raise
            
        # calculate determinant
        result_tensor = (self.my_tensor[0][0]*self.my_tensor[1][1]) - (self.my_tensor[1][0]*self.my_tensor[0][1])
        return result_tensor
    
    # This method calculate inverse of 2x2 tensor
    def tensor_inverse(self):
        # check if the tensor is compatible for inverse calcualtion, if not raise an exception
        # tensor as a square matrix with size 2x2 is compatible for inverse calculation
        # note as of now only 2x2 tensor is supported
        try:
            if(len(self.my_tensor) != 2 or len(self.my_tensor[0]) != 2):
                raise TensorNotSuitableError("Tensor not compatible for inverse!!!")
                
            # calculate determinant, In case if the determinant is 0 then raise an exception as 
            # divide by zero can cause an exception
            result_determinant = self.tensor_determinant()
            if (result_determinant == 0):
                raise TensorNotSuitableError("Tensor not compatible for inverse, determinant is zero!!!")
            
        except TensorNotSuitableError:
            raise
        
        # create result tensor 
        result_tensor = self.create_all_zero_tensor((len(self.my_tensor),len(self.my_tensor[0])))
        # swap positions of element[0[0] and element [1][1]
        temp = self.my_tensor[0][0]
        self.my_tensor[0][0] = self.my_tensor[1][1]
        self.my_tensor[1][1] = temp
        # put negatives in front of element[1][0] and element[0][1]
        self.my_tensor[1][0] *= -1
        self.my_tensor[0][1] *= -1

        # get the row of tensor 
        for row in range (len(self.my_tensor)):
            # get the column of tensor 
            for column in range(len(self.my_tensor[0])):
                result_tensor[row][column] = self.my_tensor[row][column] / result_determinant
        return result_tensor

    # This method calculate cross product of suitable tensors
    def tensor_cross_product(self,other_tensor):
        # check if the tensor is a vector with 3 rows, if not raise an exception
        # cross product is compatible only for 3 dimensions x,y,z
        try:
            if((len(self.my_tensor) != 3) or (len(self.my_tensor[0]) != 1)):
               raise TensorNotSuitableError("Tensor not compatible for cross product!!!")
            elif ((len(other_tensor) != 3) or (len(other_tensor[0]) != 1)):
             raise TensorNotSuitableError("Tensor not compatible for cross product!!!")
        except TensorNotSuitableError:
            raise

        # create result tensor 
        result_tensor = self.create_all_zero_tensor((3,1))
        # Apply the cross product rule
        result_tensor[0][0] = (self.my_tensor[1][0] * other_tensor[2][0]) - (self.my_tensor[2][0] * other_tensor[1][0])
        result_tensor[1][0] = (self.my_tensor[2][0] * other_tensor[0][0]) - (self.my_tensor[0][0] * other_tensor[2][0])
        result_tensor[2][0] = (self.my_tensor[0][0] * other_tensor[1][0]) - (self.my_tensor[1][0] * other_tensor[0][0])
        return result_tensor
    
# Class Test_etivity4
# This class includes test code to check the sanity of 
# all the tensor operations provided in class Tensor. 
class Test_etivity4(unittest.TestCase):
    def test_tensor_operations_2x2_addition(self):
        first_tensor = Tensor()
        second_tensor = Tensor()
        first_sample_tensor =  [[1,2],[2,5]]
        second_sample_tensor = [[3,5],[2,5]] 
        first_tensor.set_my_tensor(first_sample_tensor)
        second_tensor.set_my_tensor(second_sample_tensor)    
        expected_result = [[4,7],[4,10]]
        
        result = first_tensor.tensor_addition_or_subtraction(second_tensor.return_my_tensor())
        self.assertEqual(result,expected_result)
        
    def test_tensor_operations_2x2_subtraction(self): 
        first_tensor = Tensor()
        second_tensor = Tensor()
        first_sample_tensor =  [[1,5],[5,1]]
        second_sample_tensor = [[2,1],[1,2]]
        first_tensor.set_my_tensor(first_sample_tensor)
        second_tensor.set_my_tensor(second_sample_tensor)
        expected_result = [[-1,4],[4,-1]]

        result = first_tensor.tensor_addition_or_subtraction(second_tensor.return_my_tensor(),"subtraction")
        self.assertEqual(result,expected_result)
        
    def test_tensor_operations_2x2_multiplication(self):
        first_tensor = Tensor()
        second_tensor = Tensor()
        first_sample_tensor =  [[2,4],[5,5]]
        second_sample_tensor = [[1,4],[5,1]]
        first_tensor.set_my_tensor(first_sample_tensor)
        second_tensor.set_my_tensor(second_sample_tensor)
        expected_result = [[22,12],[30,25]]

        result = first_tensor.tensor_multiplication(second_tensor.return_my_tensor())
        self.assertEqual(result,expected_result)

    def test_tensor_operations_2x2_determinant(self):
        tensor = Tensor()
        first_sample_tensor =  [[2,4],[5,5]]
        tensor.set_my_tensor(first_sample_tensor)
        expected_result = -10

        result = tensor.tensor_determinant()
        self.assertEqual(result,expected_result)
        
    def test_tensor_operations_2x2_inverse(self):
        tensor = Tensor()
        first_sample_tensor =  [[2,4],[5,5]]
        tensor.set_my_tensor(first_sample_tensor)
        expected_result = [[-0.5,0.4],[0.5,-0.2]]

        result = tensor.tensor_inverse()
        self.assertEqual(result,expected_result)
        
    def test_tensor_operations_3x1_cross_product(self):
        first_tensor = Tensor()
        second_tensor = Tensor()
        first_sample_tensor =  [[1],[5],[6]]
        second_sample_tensor = [[3],[7],[6]]
        first_tensor.set_my_tensor(first_sample_tensor)
        second_tensor.set_my_tensor(second_sample_tensor)
        expected_result = [[-12],[12],[-8]]

        result = first_tensor.tensor_cross_product(second_tensor.return_my_tensor())
        self.assertEqual(result,expected_result)
        
    def test_tensor_exception_handling(self):
        tensor = Tensor()
        first_sample_tensor =  [[2,4,5],[5,5,7]]
        tensor.set_my_tensor(first_sample_tensor)

        try:
            tensor.tensor_determinant()
        except TensorNotSuitableError:
            exception_raised = True
        self.assertEqual(exception_raised,True)
        
# Main
if __name__ == "__main__":
    # create 2x2 first tensor
    first_tensor = Tensor((4,4))
    # calculate the size of the tensor as tuple and print it
    print ("First tensor size as tuple: " + str(first_tensor.calculate_tensor_size()))
    print (first_tensor.format_tensor(first_tensor.return_my_tensor()) + "\n")

    # create 2x2 second tensor
    second_tensor = Tensor((4,4))
    # calculate the size of the tensor as tuple and print it
    print ("Second tensor size as tuple: " + str(second_tensor.calculate_tensor_size()))
    print (second_tensor.format_tensor(second_tensor.return_my_tensor()) + "\n")

    # perform addition, subtraction and multiplication with first and second tensors
    print ("Result tensor for operation addition ")
    result = first_tensor.tensor_addition_or_subtraction(second_tensor.return_my_tensor())
    print(first_tensor.format_tensor(result))
    
    print ("\nResult tensor for operation subtraction ")
    result = first_tensor.tensor_addition_or_subtraction(second_tensor.return_my_tensor(), "subtraction")
    print(first_tensor.format_tensor(result))
    
    print ("\nResult tensor for operation multiplication ")
    result = first_tensor.tensor_multiplication(second_tensor.return_my_tensor())
    print(first_tensor.format_tensor(result))
    
    # create 2x2 first tensor to calculate determinant and inverse
    tensor_for_det_inv = Tensor((2,2))
    # calculate the size of the tensor as tuple and print it
    print ("\n\nTensor for Determinant and Inverse calculation, size as tuple: " + str(tensor_for_det_inv.calculate_tensor_size()))
    print (tensor_for_det_inv.format_tensor(tensor_for_det_inv.return_my_tensor()))
    # calculate determinant
    print ("\nResult for operation determinant")
    result = tensor_for_det_inv.tensor_determinant()
    print(result)
    # calculate inverse
    print ("\nResult tensor for operation inverse")
    result = tensor_for_det_inv.tensor_inverse()
    print(first_tensor.format_tensor(result))

    # create 3x1 first vector to calculate cross product
    first_tensor_for_cross_prod = Tensor((3,1))
    print ("\n\nFirst tensor for cross product calculation, size as tuple: " + str(first_tensor_for_cross_prod.calculate_tensor_size()))
    print (first_tensor_for_cross_prod.format_tensor(first_tensor_for_cross_prod.return_my_tensor()) + "\n")
    
    # create 3x1 second vector to calculate cross product
    second_tensor_for_cross_prod = Tensor((3,1))
    print ("Second tensor for cross product calculation, size as tuple: " + str(second_tensor_for_cross_prod.calculate_tensor_size()))
    print (second_tensor_for_cross_prod.format_tensor(second_tensor_for_cross_prod.return_my_tensor()) + "\n")
    
    # calculate crossproduct
    print ("Result tensor for operation cross product")
    result = first_tensor_for_cross_prod.tensor_cross_product(second_tensor_for_cross_prod.return_my_tensor())
    print(first_tensor_for_cross_prod.format_tensor(result))
    
    # Run test code
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    

.......

First tensor size as tuple: (4, 4)
7  5  1  1
3  8  8  1
3  8  3  7
5  2  1  1

Second tensor size as tuple: (4, 4)
2  3  4  8
6  4  4  3
8  7  1  7
6  2  3  8

Result tensor for operation addition 
9  8  5  9
9  12  12  4
11  15  4  14
11  4  4  9

Result tensor for operation subtraction 
5  2  -3  -7
-3  4  4  -2
-5  1  2  0
-1  0  -2  -7

Result tensor for operation multiplication 
58  50  52  86
124  99  55  112
120  76  68  125
36  32  32  61


Tensor for Determinant and Inverse calculation, size as tuple: (2, 2)
5  7
1  3

Result for operation determinant
8

Result tensor for operation inverse
0.375  -0.875
-0.125  0.625


First tensor for cross product calculation, size as tuple: (3, 1)
3
7
6

Second tensor for cross product calculation, size as tuple: (3, 1)
1
2
2

Result tensor for operation cross product
2
0
-1



----------------------------------------------------------------------
Ran 7 tests in 0.014s

OK
